1) Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

        *Наименование вакансии
        *Предлагаемую зарплату (отдельно мин. и и отдельно макс.)
        *Ссылку на саму вакансию        
        *Сайт откуда собрана вакансия
По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.


In [1]:
import re
import urllib.parse
import urllib.request as request
import pandas as pd
from typing import List

import requests
from bs4 import BeautifulSoup as bs

In [2]:
class WorkGrab:
    def __init__(self, grabbers: List['SuperJob'], search_string=""):
        self.grabbers = grabbers
        self.search_string = search_string

    def get_page(self, keywords, page=1, ):
        return self._do_request(self._url, {"page": page, "keywords": keywords})

    def __iter__(self):
        for grabber in self.grabbers:
            gr = grabber(self.search_string)
            for num in range(1, 6):
                page = gr.get_page(num)
                yield from gr.next(page.text)

In [3]:
class SuperJob:
    MAIN_URL = "https://www.superjob.ru/"
    SEARCH_URL = "/vacancy/search/"
    SOURCE = "superjob"

    def __init__(self, search_string=""):
        self._config = self._configure()
        self.search_string = search_string

    def _configure(self):
        url = request.urljoin(self.MAIN_URL, self.SEARCH_URL)
        config = {
            "main_url": self.MAIN_URL,
            "url": url
        }
        return config

    @property
    def _url(self):
        return self._config["url"]

    def get_page(self, page=1, ):
        return self._do_request(self._url, {"page": page,
                                            "keywords": self.search_string})

    def _append_search_params(self):
        params = {
            "geo[c][0]": 1
        }
        return params

    def _do_request(self, url, params: dict):
        params.update(self._append_search_params())
        return requests.get(url, params)

    @classmethod
    def _quote(cls, value: str):
        return value

    def next(self, text):
        soup = bs(text, "lxml")
        for vacancy in self._find_vacancies(soup):
            link = vacancy.find("a", {"class": ["_2JivQ", "_3dPok"]})
            salary = self._parse_salary(vacancy)

            yield {
                "text": link.text,
                "href": self._href(link),
                "salary": salary,
                "source": self.SOURCE
            }

    def _href(self, soup):
        href = soup.attrs['href']
        return urllib.parse.urljoin(self.MAIN_URL, href)

    def _find_vacancies(self, soup):
        all_vacancies = soup.select("div._2g1F->"
                                    "div._212By._37XTu>"
                                    "div._2g1F->"
                                    "div._212By._37XTu")
        return all_vacancies

    def _parse_salary(self, source):
        table = str.maketrans("от\xa0₽", "    ")
        reg = re.compile("\d+")

        salary_raw = reg.findall(
            source
                .find("span", {"class": "f-test-text-company-item-salary"})
                .text
                .translate(table)
                .strip()
        )
        return "".join(salary_raw[:2]) if salary_raw else None


In [4]:
class HeadHunter:
    MAIN_URL = "https://hh.ru"
    SEARCH_URL = "/search/vacancy"
    SOURCE = "hh"

    table = str.maketrans("от\xa0₽", "    ")
    number_reg = re.compile("\d+")

    def __init__(self, search_string=""):
        self._config = self._configure()
        self.search_string = search_string

    def _configure(self):
        url = request.urljoin(self.MAIN_URL, self.SEARCH_URL)
        config = {
            "main_url": self.MAIN_URL,
            "url": url
        }
        return config

    @property
    def _url(self):
        return self._config["url"]

    def get_page(self, page):
        page -= 1
        return self._do_request(self._url, {"page": page,
                                            "text": self.search_string})

    def _append_search_params(self):
        params = {
            "clusters": True,
            "enable_snippets": True,
            "showClusters": True,

        }
        return params

    def _do_request(self, url, params: dict):
        params.update(self._append_search_params())
        return requests.get(url, params,
                            headers={"User-Agent": "Chrome/51.0.2704.64 Safari/537.36"})

    def next(self, text):
        soup = bs(text, "lxml")
        for vacancy in self._find_vacancies(soup):
            title = vacancy.find("div", {"class": "resume-search-item__name"})
            salary = self._parse_salary(vacancy)

            yield {
                "text": title.text,
                "href": self._href(title),
                "salary": salary,
                "source": self.SOURCE
            }

    def _href(self, soup):
        href = soup.find("a").attrs['href']
        return urllib.parse.urljoin(self.MAIN_URL, href)

    def _find_vacancies(self, soup):
        all_vacancies = soup.findAll("div", {"class": "vacancy-serp-item"})
        return all_vacancies

    def _parse_salary(self, source):
        item = source.find("div", {"class": "vacancy-serp-item__compensation"})
        if item is None:
            return None

        salary_raw = self.number_reg.findall(
            item.text.translate(self.table).strip()
        )
        return "".join(salary_raw[:2]) if salary_raw else None

In [5]:
work_grab = WorkGrab(grabbers=[SuperJob, HeadHunter], search_string="Разработчик Python")

vacancies = []
for vacancy in work_grab:
    vacancies.append(vacancy)

df = pd.DataFrame(vacancies, columns=['href', "salary", "source", "text"])

print(df.head())
print(df.count())

                                                href  salary    source  \
0  https://www.superjob.ru/vakansii/python-razrab...   70000  superjob   
1  https://www.superjob.ru/vakansii/lead-razrabot...  150000  superjob   
2  https://www.superjob.ru/vakansii/razrabotchik-...    None  superjob   
3  https://www.superjob.ru/vakansii/fullstack-raz...    None  superjob   
4  https://www.superjob.ru/vakansii/backend-razra...   35000  superjob   

                           text  
0  Python-разработчик (backend)  
1          Lead-разработчик php  
2        Разработчик алгоритмов  
3     FullStack-разработчик PHP  
4           Backend разработчик  
href      145
salary     79
source    145
text      145
dtype: int64
